# Image Captioning 101

In [ ]:
!mkdir coco
!mkdir data

In [ ]:
!wget http://msvocds.blob.core.windows.net/annotations-1-0-3/captions_train-val2014.zip -P ./coco/
!wget http://images.cocodataset.org/zips/train2014.zip -P ./coco/

--2021-10-21 09:38:42--  http://msvocds.blob.core.windows.net/annotations-1-0-3/captions_train-val2014.zip
Resolving msvocds.blob.core.windows.net (msvocds.blob.core.windows.net)... 20.60.195.163
Connecting to msvocds.blob.core.windows.net (msvocds.blob.core.windows.net)|20.60.195.163|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19673183 (19M) [application/octet-stream Charset=UTF-8]
Saving to: ‘./coco/captions_train-val2014.zip’

captions_train-val2 100%[===================>]  18.76M  48.0MB/s    in 0.4s    

2021-10-21 09:38:43 (48.0 MB/s) - ‘./coco/captions_train-val2014.zip’ saved [19673183/19673183]

--2021-10-21 09:38:43--  http://images.cocodataset.org/zips/train2014.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 54.231.128.217
Connecting to images.cocodataset.org (images.cocodataset.org)|54.231.128.217|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13510573713 (13G) [application/zip]
Saving to: ‘./coco/trai

In [ ]:
!unzip ./coco/captions_train-val2014.zip -d ./coco/
!unzip ./coco/train2014.zip -d ./coco/

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
 extracting: ./coco/train2014/COCO_train2014_000000173228.jpg  
 extracting: ./coco/train2014/COCO_train2014_000000222259.jpg  
 extracting: ./coco/train2014/COCO_train2014_000000336426.jpg  
 extracting: ./coco/train2014/COCO_train2014_000000027270.jpg  
 extracting: ./coco/train2014/COCO_train2014_000000345263.jpg  
 extracting: ./coco/train2014/COCO_train2014_000000010407.jpg  
 extracting: ./coco/train2014/COCO_train2014_000000286469.jpg  
 extracting: ./coco/train2014/COCO_train2014_000000490801.jpg  
 extracting: ./coco/train2014/COCO_train2014_000000432036.jpg  
 extracting: ./coco/train2014/COCO_train2014_000000349139.jpg  
 extracting: ./coco/train2014/COCO_train2014_000000570628.jpg  
 extracting: ./coco/train2014/COCO_train2014_000000572717.jpg  
 extracting: ./coco/train2014/COCO_train2014_000000557272.jpg  
 extracting: ./coco/train2014/COCO_train2014_000000093587.jpg  
 extracting: ./coco/train2014/COCO_train2014_000000529864.jpg  
 ext

In [ ]:
!rm ./coco/captions_train-val2014.zip
!rm ./coco/train2014.zip

In [ ]:
import os
import pickle
import numpy as np
import nltk
import torch
import torchvision.transforms as transforms
import torch.utils.data as data
import torch.nn as nn
import matplotlib.pyplot as plt

from multiprocessing import cpu_count
from collections import Counter
from PIL import Image
from pycocotools.coco import COCO
from torch.nn.utils.rnn import pack_padded_sequence
from torchvision.models import resnet152

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
def resize_images(image_dir, output_dir, size):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    images = os.listdir(image_dir)
    num_images = len(images)
    for i, image in enumerate(images):
        with open(os.path.join(image_dir, image), 'r+b') as f:
            with Image.open(f) as img:
                img = img.resize(size, Image.ANTIALIAS)
                img.save(os.path.join(output_dir, image), img.format)
        if (i + 1) % 10000 == 0:
            print ("[{}/{}] Image Resized".format(i + 1, num_images))

In [ ]:
image_dir = './coco/train2014/'
output_dir = './data/train2014/'
image_size = [256, 256]

resize_images(image_dir, output_dir, image_size)

[10000/36278] Image Resized
[20000/36278] Image Resized
[30000/36278] Image Resized


OSError: ignored

In [ ]:
class Vocabulary(object):
    def __init__(self):
        self.word2idx = {}
        self.idx2word = {}
        self.idx = 0

    def add_word(self, word):
        if not word in self.word2idx:
            self.word2idx[word] = self.idx
            self.idx2word[self.idx] = word
            self.idx += 1

    def __call__(self, word):
        if not word in self.word2idx:
            return self.word2idx['<unk>']
        return self.word2idx[word]

    def __len__(self):
        return len(self.word2idx)

In [ ]:
def build_vocab(json, threshold):
    dataset = COCO(json)
    keys = dataset.anns.keys()
    counter = Counter()

    for i, k in enumerate(keys):
        caption = str(dataset.anns[k]['caption'])
        tokens = nltk.tokenize.word_tokenize(caption.lower())
        counter.update(tokens)

        if (i + 1) % 10000 == 0:
            print("[{}/{}] Caption Tokenized".format(i + 1, len(keys)))

    words = [word for word, cnt in counter.items() if cnt >= threshold]

    vocab = Vocabulary()
    vocab.add_word('<pad>')
    vocab.add_word('<start>')
    vocab.add_word('<end>')
    vocab.add_word('<unk>')

    for w in words:
        vocab.add_word(w)

    return vocab

In [ ]:
def build_vocab(datas, threshold):
    dataset = COCO(json)
    keys = dataset.anns.keys()
    counter = Counter()

    for i, k in enumerate(keys):
        caption = datas[]
        tokens = nltk.tokenize.word_tokenize(caption.lower())
        counter.update(tokens)

        if (i + 1) % 10000 == 0:
            print("[{}/{}] Caption Tokenized".format(i + 1, len(keys)))

    words = [word for word, cnt in counter.items() if cnt >= threshold]

    vocab = Vocabulary()
    vocab.add_word('<pad>')
    vocab.add_word('<start>')
    vocab.add_word('<end>')
    vocab.add_word('<unk>')

    for w in words:
        vocab.add_word(w)

    return vocab

In [ ]:
vocab = build_vocab('./coco/annotations/captions_train2014.json', 4)

with open('./data/vocab.pkl', 'wb') as f:
    pickle.dump(vocab, f)

In [ ]:
# !7z a myzip ./data/*

In [ ]:
class CocoDS(data.Dataset):
    def __init__(self, root, json, vocab, transform=None):
        self.root = root
        self.coco = COCO(json)
        self.keys = list(self.coco.anns.keys())
        self.vocab = vocab
        self.transform = transform

    def __getitem__(self, index):
        key = self.keys[index]
        caption = self.coco.anns[key]['caption']
        img_id = self.coco.anns[key]['image_id']
        path = self.coco.loadImgs(img_id)[0]['file_name']

        image = Image.open(os.path.join(self.root, path)).convert('RGB')
        if self.transform is not None:
            image = self.transform(image)

        tokens = nltk.tokenize.word_tokenize(str(caption).lower())
        caption = []
        caption.append(self.vocab('<start>'))
        caption.extend([self.vocab(token) for token in tokens])
        caption.append(self.vocab('<end>'))
        target = torch.Tensor(caption)
        return image, target

    def __len__(self):
        return len(self.keys)

In [ ]:
with open('./data/vocab.pkl', 'rb') as f:
    vocab = pickle.load(f)

transform = transforms.Compose([
    transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

train_dataset = CocoDS('./data/train2014/', './coco/annotations/captions_train2014.json', vocab=vocab, transform=transform)

In [ ]:
def collate_fn(data):
    """
    Define mini-batch
    data: list of tuple (image, caption) 
    Return:
        images: tensor (batch_size, 3, 256, 256).
        targets: tensor (batch_size, padded_length)
        lengths: list (valid length for each padded caption)
    """
    data.sort(key=lambda x: len(x[1]), reverse=True)    # Sort by caption length (descending)
    images, captions = zip(*data)

    images = torch.stack(images, 0)

    lengths = [len(cap) for cap in captions]
    targets = torch.zeros(len(captions), max(lengths)).long()
    for i, cap in enumerate(captions):
        end = lengths[i]
        targets[i, :end] = cap[:end]        
    return images, targets, lengths

In [ ]:
if cpu_count() > 5:
    num_workers = cpu_count() // 2
elif cpu_count() < 2:
    num_workers = 0
else:
    num_workers = 2

# Return: (image, caption, length)
### image: tensor (batch_size, 3, 224, 224).
### caption: tensor (batch_size, padded_length).
### length: list (batch_size)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=num_workers, collate_fn=collate_fn)

In [ ]:
class EncoderCNN(nn.Module):
    def __init__(self):
        super().__init__()
        pretrained = resnet152(pretrained=True)    # Get pretrained model
        layers = list(pretrained.children())[:-1]    # Delete the last FC layer
        self.resnet = nn.Sequential(*layers)
        self.linear = nn.Linear(pretrained.fc.in_features, 256)    # Create an intermediate FC layer
        self.bn = nn.BatchNorm1d(256, momentum=0.01)
        
    def forward(self, x):
        with torch.no_grad():
            features = self.resnet(x)
        features = features.view(features.size(0), -1)
        latent_vector = self.bn(self.linear(features))
        return latent_vector

In [ ]:
class DecoderLSTM(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, 256)
        self.lstm = nn.LSTM(256, 512, 1, batch_first=True)    # Single layer LSTM
        self.linear = nn.Linear(512, vocab_size)
        
    def forward(self, features, captions, lengths):
        embeddings = self.embed(captions)
        embeddings = torch.cat((features.unsqueeze(1), embeddings), 1)
        packed = pack_padded_sequence(embeddings, lengths, batch_first=True) 
        hiddens, _ = self.lstm(packed)
        outputs = self.linear(hiddens[0])
        return outputs

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

encoder = EncoderCNN().to(device)
decoder = DecoderLSTM(len(vocab)).to(device)

criterion = nn.CrossEntropyLoss()
params = list(decoder.parameters()) + list(encoder.linear.parameters()) + list(encoder.bn.parameters())
optimizer = torch.optim.Adam(params, lr=0.001)

In [ ]:
EPOCHS = 2
total_step = len(train_loader)

encoder.train()
decoder.train()

for epoch in range(EPOCHS):
    for i, (images, captions, lengths) in enumerate(train_loader):
        images, captions = images.to(device), captions.to(device)
        targets = pack_padded_sequence(captions, lengths, batch_first=True)[0]
        optimizer.zero_grad()
        
        features = encoder(images)
        outputs = decoder(features, captions, lengths)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        if (i + 1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch + 1, EPOCHS, i + 1, total_step, loss.item()))

In [ ]:
def load_image(image_path):
    img = Image.open(image_path).convert('RGB')
    img = img.resize([224, 224], Image.LANCZOS)
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
    ])
    img = transform(img).unsqueeze(0)
    return img

In [ ]:
image_path = './sample/newimage.jpg'
img = load_image(image_path).to(device)

In [ ]:
max_sequence_length = 20

encoder.eval()
decoder.eval()

word_ids = []

with torch.no_grad():
    inputs = encoder(img).unsqueeze(1)
    states = None
    for i in range(max_sequence_length):
        hiddens, states = decoder.lstm(inputs, states)
        outputs = decoder.linear(hiddens.squeeze(1))
        _, predicted = outputs.max(1)
        word_ids.append(predicted)
        inputs = decoder.embed(predicted).unsqueeze(1)

    word_ids = torch.stack(word_ids, 1)
    word_ids = word_ids[0].detach().cpu().numpy()

caption = []
for id in word_ids:
    word = vocab.idx2word[id]
    caption.append(word)
    if word == '<end>':
        break
caption = ' '.join(caption)

In [ ]:
print(caption)
image = Image.open(image_path)
plt.imshow(np.asarray(image))